<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/OpenAI_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# API を利用したAIプログラミング

## API とは
APIは、AIベンダーが提供する人工知能のAPI（Application Programming Interface）です。このAPIを使用することで、開発者は自然言語処理タスクや他のAI関連のタスクを実行するために、AIモデルを利用することができます。

**APIの利用には料金がかかります。**料金はAPIの使用量に基づいており、APIのエンドポイントの呼び出し回数や生成されたテキストの量に応じて請求されます。詳細な料金体系については、各ベンダの公式ウェブサイトや料金ページを参照ください。

## OpenAI APIでできること

ここでは、OpenAIが提供するAPIを用います．OpenAI APIは、言語生成（text generation）、文章の要約（text summarization）、質問応答（question answering）、文章翻訳（text translation）など、さまざまなタスクに対応しています。これにより、開発者はさまざまな自然言語処理のニーズに応じてAPIを利用することができます。

## APIを利用するメリット

APIを利用することで、開発者は高度な自然言語処理タスクを実行するための大規模な学習データと強力なモデルにアクセスすることができます。また、APIの利用は簡単で柔軟であり、既存のアプリケーションやサービスに統合することも容易です。

## ライブラリのインストール

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install openai gTTs pydub

## ライブラリのインポート

In [ ]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from gtts import gTTS
import IPython.display
from pydub import AudioSegment
import IPython.display
import openai
from openai import OpenAI
import os
os.environ['OPENAI_API_KEY'] = ""#@param{type:"string"}

## 0. 準備

画面の端まで行ったら改行させる設定

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

record -- マイクで録音できる関数

In [ ]:
RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec, filename='voice.wav'):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec * 1000))
  b = b64decode(s.split(',')[1])
  with open(filename, 'wb+') as f:
    f.write(b)
  # 以下3行は講義室用に必要
  sound = AudioSegment.from_file("voice.wav")
  sound.export("voice.mp3", format="mp3")

マイクで録音：結果は， voice.mp3 として保存されます．

In [ ]:
sec = 10 # 録音秒数
print(sec, "秒以内で喋ってください")
record(sec)
IPython.display.Audio("voice.mp3", autoplay=True)


## 1. 音声認識

whisper というモデルを使用します．

In [ ]:
client = OpenAI()

audio_file = open("voice.mp3", "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file
)

print(transcript.text)

In [ ]:
transcript.text

## 2. 音声合成

input 部分に言わせたいセリフを入れる

In [ ]:
speech_file_path = "speech.mp3"
response = openai.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input="The quick brown fox jumped over the lazy dog."
)
response.stream_to_file(speech_file_path)
IPython.display.Audio("speech.mp3", autoplay=True)

先ほど聞き取った内容でも

In [ ]:
speech_file_path = "speech.mp3"
response = openai.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input=transcript.text
)
response.stream_to_file(speech_file_path)
IPython.display.Audio("speech.mp3", autoplay=True)

## ChatGPT との会話
注：APIを利用する場合には，会話データは収集されません（重要）

（１） 一度に答える場合

In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "あなたは有能なアシスタントです．"},
    {"role": "user", "content": "私はこの先どのように生きていったらいいでしょうか？"}
  ]
)

text = completion.choices[0].message.content
print(text)

（２）　徐々に答える場合 --> stream=True, chunk の利用

In [ ]:
from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "あなたは有能なアシスタントです．"},
      {"role": "user", "content": "私はこの先どのように生きていったらいいでしょうか？"}
    ],
    stream=True,
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

# 課題
マイクで喋った内容を ChatGPT に入力，得られた回答を音声で喋ってくれるプログラムを書け．